# Build the Baseball Database from the core source files

## Preamble - Imports and Function Definitions

In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine
import re

In [2]:
def createDatabase(fileName):
    """
    Create a SQLite database and return the connection to it
    :param fileName: string filename of the database to connect
    :return conn: sqlite connection object
    """
    # delete database if exists
    try:
        os.remove(fileName)
    except FileNotFoundError as err:
        print('No existing database to delete')
    except Exception as err:
        print('Database deletion error: %s'%err)
        raise err
    else:
        print('Existing database deleted')
    
    # connect to the file, which creates the db
    conn = None
    try:
        conn = sqlite3.connect(fileName)
    except Error as err:
        print('Database creation error: %s'%err)
        raise err
    else:
        print('Database created')

    return conn

## Data Processing

In [3]:
# create the database
dbFile = '../data/baseballdata.db'
dbConn = createDatabase(dbFile)

Existing database deleted
Database created


In [4]:
''' manually set some things '''
# define primary keys
PKs = {'TeamsHalf.csv':['_yearID', '_teamID', '_Half'],
       'TeamsFranchises.csv':['_franchID'],
       'Teams.csv':['_yearID', '_teamID', '_franchID'],
       'SeriesPost.csv':['_yearID', '_round'],
       'Schools.csv':['_schoolID'],
       'Salaries.csv':['_yearID', '_teamID', '_lgID', '_playerID'],
       'PitchingPost.csv':['_playerID', '_yearID', '_round', '_teamID'],
       'Pitching.csv':['_playerID', '_yearID', '_stint', '_teamID'],
       'People.csv':['_playerID'],
       'Parks.csv':['_parkID'],
       'ManagersHalf.csv':['_playerID', '_yearID', '_teamID', '_half'],
       'Managers.csv':['_playerID', '_yearID', '_teamID', '_inseason'],
       'HomeGames.csv':['_yearID', '_teamID', '_parkID'],
       'HallOfFame.csv':['_playerID', '_yearID', '_votedBy'],
       'FieldingPost.csv':['_playerID', '_yearID', '_teamID', '_round', '_POS'],
       'FieldingOFsplit.csv':['_playerID', '_yearID', '_teamID', '_stint', '_POS'],
       'FieldingOF.csv':['_playerID', '_yearID', '_stint'],
       'Fielding.csv':['_playerID', '_yearID', '_stint', '_teamID', '_POS'],
       'CollegePlaying.csv':['_playerID', '_schoolID', '_yearID'],
       'BattingPost.csv':['_yearID', '_round', '_playerID', '_teamID'],
       'Batting.csv':['_playerID', '_yearID', '_stint', '_teamID'],
       'AwardsSharePlayers.csv':['_awardID', '_yearID', '_playerID'],
       'AwardsShareManagers.csv':['_awardID', '_yearID', '_playerID'],
       'AwardsPlayers.csv':['_playerID', '_awardID', '_yearID', '_lgID'],
       'AwardsManagers.csv':['_playerID', '_awardID', '_yearID'],
       'Appearances.csv':['_yearID', '_teamID', '_playerID'],
       'AllstarFull.csv':['_playerID', '_yearID', '_gameID', '_teamID', '_lgID']}

# define foreign key columns
FKLookup = {'_playerID':'People', '_franchID':'TeamsFranchises', '_teamID':'Teams',
            '_schoolID':'Schools', '_parkID':'Parks'}
FKs = {'TeamsHalf.csv':['_teamID'],
       'TeamsFranchises.csv':[],
       'Teams.csv':['_franchID'],
       'SeriesPost.csv':[],
       'Schools.csv':[],
       'Salaries.csv':['_teamID', '_playerID'],
       'PitchingPost.csv':['_playerID', '_teamID'],
       'Pitching.csv':['_playerID', '_teamID'],
       'People.csv':[],
       'Parks.csv':[],
       'ManagersHalf.csv':['_playerID', '_teamID'],
       'Managers.csv':['_playerID', '_teamID'],
       'HomeGames.csv':['_teamID', '_parkID'],
       'HallOfFame.csv':['_playerID'],
       'FieldingPost.csv':['_playerID', '_teamID'],
       'FieldingOFsplit.csv':['_playerID', '_teamID'],
       'FieldingOF.csv':['_playerID'],
       'Fielding.csv':['_playerID', '_teamID'],
       'CollegePlaying.csv':['_playerID', '_schoolID'],
       'BattingPost.csv':['_playerID', '_teamID'],
       'Batting.csv':['_playerID', '_teamID'],
       'AwardsSharePlayers.csv':['_playerID'],
       'AwardsShareManagers.csv':['_playerID'],
       'AwardsPlayers.csv':['_playerID'],
       'AwardsManagers.csv':['_playerID'],
       'Appearances.csv':['_teamID', '_playerID'],
       'AllstarFull.csv':['_playerID', '_teamID']}

In [5]:
''' create the tables in the database '''
# create the types map - anything else will be text
types = {'int64':'integer', 'float64':'real', 'object':'text'}

# regexp to get allowed chars
reg = re.compile('[^a-zA-Z0-9_]')

# get the list of files and the tablenames that will be generated
corePath = '../baseballdatabank/core/'
coreFiles = {f:[f[:-4], None] for f in os.listdir(corePath) if f.endswith('.csv')}

# iterate over the files
errCnt = 0
for file in coreFiles.keys():
    print('Processing %s'%file)
    # load the data
    data = pd.read_csv(corePath+os.sep+file)
    # prefix all columns with "_", to most easily handle column names starting with numbers,
    # replace "".key" with "ID", and strip nonalphanumerics
    data = data.add_prefix('_')
    reps = {c:(c[:-4]+'ID') for c in data.columns if c[-4:]=='.key'}
    if reps:
        print('\tRenaming columns: %s'%reps)
        data.rename(columns=reps, inplace=True)
    reps = {c:reg.sub('', c) for c in data.columns if reg.search(c)}
    if reps:
        print('\tRenaming columns: %s'%reps)
        data.rename(columns=reps, inplace=True)
    # save the data for loading
    coreFiles[file][1] = data.drop_duplicates(inplace=False)
    # get the PK columns
    pks = ', '.join(PKs[file])
    # get the FK relationships
    fks = ', '.join(['FOREIGN KEY (%s) REFERENCES %s (%s)'%(val, FKLookup[val], val) for val in FKs[file]]) + ');'
    # create the create table statement
    sql = 'CREATE TABLE %s ('%coreFiles[file][0] + ', '.join([' '.join((c, types.get(d.name, 'text'))) for (c, d) in zip(data.columns, data.dtypes)]) + ', PRIMARY KEY(%s)'%pks + fks
    # TODO: need to add in the table relationships - FOREIGN KEY (project_id) REFERENCES projects (id)
    print('\t%s'%sql)
    # execute the create table statement after dropping the table
    try:
        dbConn.cursor().execute('DROP TABLE IF EXISTS %s;'%coreFiles[file][0])
    except Error as err:
        print('\tDropping table error: %s'%err)
    try:
        dbConn.cursor().execute(sql)
    except Error as err:
        print('\tCreating table error: %s'%err)
        errCnt += 1

if errCnt > 0:
    print('%d table creation errors - database may be in an inconsistent state!'%errCnt)

Processing AllstarFull.csv
	CREATE TABLE AllstarFull (_playerID text, _yearID real, _gameNum real, _gameID text, _teamID text, _lgID text, _GP integer, _startingPos real, PRIMARY KEY(_playerID, _yearID, _gameID, _teamID, _lgID)FOREIGN KEY (_playerID) REFERENCES People (_playerID), FOREIGN KEY (_teamID) REFERENCES Teams (_teamID));
Processing Appearances.csv
	CREATE TABLE Appearances (_yearID integer, _teamID text, _lgID text, _playerID text, _G_all integer, _GS real, _G_batting integer, _G_defense real, _G_p integer, _G_c integer, _G_1b integer, _G_2b integer, _G_3b integer, _G_ss integer, _G_lf integer, _G_cf integer, _G_rf integer, _G_of integer, _G_dh real, _G_ph real, _G_pr real, PRIMARY KEY(_yearID, _teamID, _playerID)FOREIGN KEY (_teamID) REFERENCES Teams (_teamID), FOREIGN KEY (_playerID) REFERENCES People (_playerID));
Processing AwardsManagers.csv
	CREATE TABLE AwardsManagers (_playerID text, _awardID text, _yearID integer, _lgID text, _tie text, _notes text, PRIMARY KEY(_play

In [6]:
''' create a new connection with sqlalchemy to allow bulk storing data from pandas '''
# close this connection
try:
    dbConn.close()
except Error as err:
    pass

# create the new connection
try:
    engine = create_engine('sqlite://%s'%dbFile[1:], echo=False)
    dbConn = engine.connect()
    print('Connected')
except Exception as err:
    print('Connection error: %s'%err)

Connected


In [7]:
''' load the data from the core source files '''
errCnt = 0
for file in coreFiles.keys():
    print('Processing %s'%file)
    try:
        coreFiles[file][1].to_sql(coreFiles[file][0], dbConn, if_exists='append', index=False)
    except Exception as err:
        print('\tPopulating table error: %s'%err)
        errCnt += 1

if errCnt > 0:
    print('%d table population errors - database may be in an inconsistent state!'%errCnt)

Processing AllstarFull.csv
Processing Appearances.csv
Processing AwardsManagers.csv
Processing AwardsPlayers.csv
Processing AwardsShareManagers.csv
Processing AwardsSharePlayers.csv
Processing Batting.csv
Processing BattingPost.csv
Processing CollegePlaying.csv
Processing Fielding.csv
Processing FieldingOF.csv
Processing FieldingOFsplit.csv
Processing FieldingPost.csv
Processing HallOfFame.csv
Processing HomeGames.csv
Processing Managers.csv
Processing ManagersHalf.csv
Processing Parks.csv
Processing People.csv
Processing Pitching.csv
Processing PitchingPost.csv
Processing Salaries.csv
Processing Schools.csv
Processing SeriesPost.csv
Processing Teams.csv
Processing TeamsFranchises.csv
Processing TeamsHalf.csv


In [8]:
# close the connection
dbConn.close()